In [1]:
from ovcg import *
from fesol import *
from pyccx import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor
from mapper import generate_mapper
from pyccx.logger import logger
import logging

In [2]:
k=1
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

logger.basicConfig(level=0)

use_relax = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFPSP_avg%i.pvd' % k)
fnodes = solverF.get_coordinates_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)

In [4]:
# solid solver
jobname = 'solid/solidD'
output = 'asc'
prob = Problem(jobname=jobname, output=output)
msolver = prob.get_solver()
msolver.initailize()
# since we know there is one step
# we stop b4 it
msolver.solve(stopb4step=1)
nlgeom = msolver.get_nlgeomsolver()
nlgeom.initialize()

# initialize our interface set
iset = nlgeom.get_set(
    set_name='Interface',
    set_type='sfelem',
    is_surface=True
)
nset = nlgeom.get_set(
    set_name='Inodes',
    set_type='node',
    is_surface=True
)

# retrieve geometry data
snodes = nset.coordinates()
scents = iset.extract_face_centers()

# create cht adapter
solverS = CHTAdapter(nlgeom=nlgeom)
solverS.add_interface(set_acc=nset, is_settable=True, itype=CHTAdapter.TEMP)
solverS.add_interface(set_acc=iset, is_settable=False, itype=CHTAdapter.FILM)

# now we are ready

INFO - mastersolver: CCX jobname: solid/solidD, output format: asc.
INFO - mastersolver: Calling LIBCCX_solverInit...
INFO - mastersolver: LIBCCX_solverInit done.
INFO - mastersolver: Solving the problem until step 1.
INFO - mastersolver: And, done!
INFO - mastersolver: Generating the nonlinear geometrical solver...
INFO - nlgeomsolver: Calling LIBCCX_stepInit...
INFO - nlgeomsolver: LIBCCX_stepInit done.
INFO - genericaccessor: Set name: Interface, set id: 4, set type: T.
INFO - genericaccessor: Updating or getting topology data of set Interface.
INFO - genericaccessor: Set size: 264.
INFO - genericaccessor: Updating or getting connectivity...
INFO - genericaccessor: Finish getting topology data for set Interface.
INFO - genericaccessor: Set name: Inodes, set id: 3, set type: N.
INFO - genericaccessor: Updating or getting topology data of set Inodes.
INFO - genericaccessor: Set size: 171.
INFO - genericaccessor: Finish getting topology data for set Inodes.
INFO - chtadapter: Setting i

In [5]:
Tf1 = 'Tf1'
Ts1 = 'Ts1'
hf = 'hf'
hs = 'hs'
Tf = 'Tf'
Ts = 'Ts'
Ff2Fs = {Tf:Ts}
Ts2Tf = {Ts1:Tf1,hs:hf}
_, s2f_mapper = generate_mapper(fnodes, scents, {}, Ts2Tf,r1=0.05,r2=0.2)
f2s_mapper, _ = generate_mapper(fnodes, snodes, Ff2Fs, {},r1=0.05,r2=0.2)

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=fnodes.shape[0], value=300.0)
abTempS = InterfaceData(size=scents.shape[0], value=310.0)
abTempS2F = InterfaceData(size=fnodes.shape[0], value=310.0)
tempF2S = InterfaceData(size=fnodes.shape[0], value=300.0)
tempS = InterfaceData(size=snodes.shape[0], value=310.0)
tempF = InterfaceData(size=fnodes.shape[0], value=300.0)
thetaT = InterfaceData(size=fnodes.shape[0], value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 300

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
hS = solverS['Interface', GET].get_sink_and_film()[1]

# map the coeff to fluid
s2f_mapper.put(hs, hS)
s2f_mapper.apply(hs, hf)
hS2F =s2f_mapper.get(hf)

CF1 = hS2F/(hS2F+hF)
CF2 = hF/(hS2F+hF)

INFO - chtapp: Extracting film and sink...
INFO - chtapp: And, done!


In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
t = 0.0
Nmax = 500
avg_pc_iterations = 0.0
flog = open('FPSP_avg%i.log' % k, mode='w')
flog.write('Fluid pseudo Robin with solid pseudo Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FPSP_avg%i.log' % k, mode='a')

abtempS_prev = abTempS2F.curr.copy()
abtempF_prev = abTempF.curr.copy()

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup_state()
    
    abtempS_prev[:] = abTempS2F.curr
    abtempF_prev[:] = abTempF.curr
    
    # mapping
    s2f_mapper.put(Ts1, abTempS.curr)
    s2f_mapper.apply(Ts1, Tf1)
    abTempS2F.curr[:] = s2f_mapper.get(Tf1)
    tempF.curr[:] = CF1*abTempS2F.curr + CF2*abTempF.curr
    
    solverF.put_temperature_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=tempF.curr
    )
    
    while True:
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # compute wall temperature in fluid and map to solid
        # since we don't have routine to project face center values
        # onto nodes, do this right now
        tempF2S.curr[:] = 0.5*(CF1*(abTempS2F.curr+abtempS_prev) + CF2*(abTempF.curr+abtempF_prev))
        
        # mapping
        f2s_mapper.put(Tf, tempF2S.curr)
        f2s_mapper.apply(Tf, Ts)
        tempS.curr[:] = f2s_mapper.get(Ts)
        # update solid flux interface
        solverS['Inodes', SET].set_temperatures(tempS.curr)
        
        # advance solid
        solverS.adjust_timesize(dt)
        solverS.increment()
        # retrieve solid interface temperature
        abTempS.curr[:] = solverS['Interface', GET].get_sink_and_film()[0]
        
        # mapping
        s2f_mapper.put(Ts1, abTempS.curr)
        s2f_mapper.apply(Ts1, Tf1)
        abTempS2F.curr[:] = s2f_mapper.get(Tf1)
        tempF.curr[:] = CF1*abTempS2F.curr + CF2*abTempF.curr
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverS.finish_increment()
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF)
                under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=0.5*(tempF.curr+CF1*abtempS_prev + CF2*abtempF_prev)
            )
            solverF.restore_previous_solutions()
            solverS.restore_state()
            solverS.finish_increment()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        fluid_ofile.write(solverF, t)
flog.close()

nlgeom.finalize()
msolver.solve(skipsolve=True)
msolver.finalize()

INFO - chtadapter: Backup current state...
INFO - chtadapter: Backup finished.
INFO - chtapp: Assign temperatures...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting film and sink...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtapp: Assign temperatures...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting film and sink...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing

INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtapp: Assign temperatures...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting film and sink...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtapp: Assign temperatures...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting film and sink...
INFO - chtapp: A

INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting film and sink...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtapp: Assign temperatures...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting film and sink...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtapp: Assign temperatures...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsol

step=0, pc_iterations=44.


INFO - chtadapter: Backup current state...
INFO - chtadapter: Backup finished.
INFO - chtapp: Assign temperatures...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting film and sink...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtapp: Assign temperatures...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting film and sink...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing

...

INFO - chtapp: Assign temperatures...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting film and sink...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtadapter: Backup current state...
INFO - chtadapter: Backup finished.


step=498, pc_iterations=0.


INFO - chtapp: Assign temperatures...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting film and sink...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!


step=499, pc_iterations=0.


INFO - nlgeomsolver: Shutting down the solver...
INFO - mastersolver: Cleaning the leftover steps from step 1.
INFO - mastersolver: And, done!
INFO - mastersolver: Shutting down the solver...


In [9]:
theta=(solverF.get_temperature_interface(3,0,1)[0:100]-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFPSP_avg%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.35893212 0.37170317 0.38360378 0.39497363 0.40588292 0.41636463
 0.42644507 0.43614775 0.44549263 0.45449586 0.46317024 0.47152604
 0.47957229 0.48731805 0.49477369 0.50195215 0.50886903 0.51554126
 0.52198478 0.528213   0.53423723 0.54006788 0.54571497 0.55118812
 0.55649646 0.56164849 0.56665198 0.57151392 0.57624057 0.58083759
 0.58531018 0.58966312 0.59390086 0.59802752 0.60204687 0.60596241
 0.6097774  0.61349488 0.61711772 0.62064852 0.62408966 0.62744326
 0.63071122 0.63389535 0.63699732 0.6400187  0.64296089 0.6458251
 0.64861235 0.65132351 0.65395933 0.65652046 0.65900742 0.66142057
 0.66376018 0.66602644 0.66821946 0.67033931 0.67238589 0.67435896
 0.67625806 0.67808257 0.6798317  0.68150434 0.68309902 0.68461385
 0.68604667 0.68739527 0.68865743 0.68983092 0.69091338 0.69190226
 0.69279493 0.69358883 0.69428156 0.69487077 0.69535393 0.69572805
 0.6959897  0.69613507 0.69616    0.69605985 0.69582887 0.69545967
 0.69494331 0.69427045 0.6934326  0.69242282 0.69123558 0.68986

In [10]:
hS2F

array([225050.50590509, 211555.3194591 , 195633.15551465, 179698.89390238,
       165351.5249582 , 153530.07497421, 144612.47228341, 138528.90851148,
       134907.62613512, 133222.2557842 , 132931.00797448, 133576.72819375,
       134817.19615858, 136391.01716766, 138066.28077394, 139616.30490489,
       140852.44267672, 141690.61110319, 142182.20563656, 142459.47821931,
       142639.53864554, 142769.88552509, 142847.90533334, 142870.67199677,
       142863.80199327, 142864.95772595, 142889.3471493 , 142917.50322066,
       142920.46616079, 142893.42887418, 142862.98794069, 142859.78018832,
       142886.28098421, 142913.95606671, 142913.40732208, 142885.11952783,
       142858.85839042, 142862.40532816, 142891.96934142, 142916.35879771,
       142909.82337774, 142879.30761804, 142856.91902927, 142866.41689255,
       142897.66278486, 142917.75342028, 142905.43083669, 142873.77210933,
       142856.07456546, 142871.1704241 , 142902.97656668, 142918.03391254,
       142900.33807344, 1

In [11]:
hS

array([107978.02197794, 125171.97452229, 125171.97452265, 148878.78787873,
       286759.3778883 , 148878.78787873, 107978.02197794, 272000.00000002,
       237239.78478475, 160338.54085426, 197270.40776026, 107978.02197831,
       148878.78787873, 147410.19530421, 152951.5270158 , 176839.05245964,
       148878.78787873, 148878.78787873, 148452.75916766, 111690.9204416 ,
       148452.75916766, 214556.02396854, 148878.78787873, 125171.97452265,
       125171.97452229, 125171.97452229, 148878.78787873, 107978.02197795,
       148878.78787873, 125171.97452215, 125171.97452229, 125171.97452215,
       125171.97452215, 148878.78787873, 148878.78787873, 113137.57994313,
       125171.97452229, 125171.97452215, 107978.02197794, 109083.27863515,
       147410.19530431, 214391.77847105, 125171.97452229, 124483.45602817,
       125171.97452265, 207588.29666739, 190281.46745309, 148878.78787873,
       237239.78478473, 157910.0145138 , 197270.40775997, 125318.10037452,
       108528.05252584, 1